In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics, linear_model
from sklearn.ensemble import RandomForestClassifier

# Loading the data

In [2]:
studentLogs = pd.concat([
    pd.read_csv('Data/student_log_'+str(i)+'.csv') for i in range(1,11)
], ignore_index=True)

trainLabels = pd.read_csv('Data/training_label.csv', index_col='ITEST_id')
trainLabels.drop_duplicates(subset=None, keep='first', inplace=True)

testLabels = pd.read_csv('Data/validation_test_label.csv', index_col='ITEST_id')

/Users/louisfaucon/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
print(studentLogs.shape)
studentLogs.head()

(942816, 77)


,AveCarelessness,AveCorrect,AveKnow,AveResBored,AveResConf,AveResEngcon,AveResFrust,AveResGaming,AveResOfftask,ITEST_id,...,timeOver80,timeSinceSkill,timeTaken,totalFrAttempted,totalFrPastWrongCount,totalFrPercentPastWrong,totalFrSkillOpportunities,totalFrSkillOpportunitiesByScaffolding,totalFrTimeOnSkill,totalTimeByPercentCorrectForskill
0,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,49.0,0,0,0.0,0,0.0,0.0,0.000000
1,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,4.0,1,0,0.0,1,0.0,49.0,106.000000
2,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,6.0,2,0,0.0,0,0.0,0.0,0.000000
3,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,18.0,3,1,0.0,1,0.0,0.0,0.000000
4,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,8,...,0,0.0,2.0,3,1,1.0,1,1.0,6.0,77.999999


In [4]:
print(trainLabels.shape)
trainLabels.head()

(467, 4)


,SchoolId,AveCorrect,MCAS,isSTEM
ITEST_id,,,,
9,2,0.438492,32,1
27,1,0.348837,21,0
33,2,0.686391,52,0
35,2,0.379658,34,0
37,3,0.305785,-999,0


In [5]:
print(testLabels.shape)
testLabels.head()

(172, 3)


,SchoolId,AveCorrect,MCAS
ITEST_id,,,
9,2,0.438492,32
101,4,0.403553,29
161,1,0.483425,40
164,2,0.256983,9
176,2,0.575949,50


# Feature engineering

In [13]:
studentsFeatures = studentLogs.groupby('ITEST_id').mean()
studentsFeatures = studentsFeatures.fillna(0)
print(studentsFeatures.shape)
studentsFeatures.head()

(1709, 71)


,AveCarelessness,AveCorrect,AveKnow,AveResBored,AveResConf,AveResEngcon,AveResFrust,AveResGaming,AveResOfftask,NumActions,...,timeOver80,timeSinceSkill,timeTaken,totalFrAttempted,totalFrPastWrongCount,totalFrPercentPastWrong,totalFrSkillOpportunities,totalFrSkillOpportunitiesByScaffolding,totalFrTimeOnSkill,totalTimeByPercentCorrectForskill
ITEST_id,,,,,,,,,,,,,,,,,,,,,
8,0.183276,0.483902,0.352416,0.208389,0.115905,0.679126,0.112408,0.196561,0.156503,1056.0,...,0.049242,1.208484e+06,22.264205,317.567235,1.254735,0.207548,6.567235,1.149585,202.714962,835.534599
9,0.099734,0.438492,0.185138,0.277149,0.098078,0.644744,0.162771,0.005554,0.213378,504.0,...,0.138889,5.742044e+05,42.494048,127.646825,1.234127,0.236869,5.269841,0.883345,311.454365,1639.285807
11,0.090318,0.326957,0.124832,0.262076,0.147169,0.643418,0.118977,0.083459,0.208981,575.0,...,0.132174,5.748212e+05,46.958261,126.706087,1.092174,0.288723,4.066087,0.914445,446.502609,2712.019060
25,0.165527,0.476718,0.284904,0.261453,0.137153,0.660054,0.087906,0.016774,0.227680,451.0,...,0.135255,6.393416e+05,42.835920,132.569845,1.199557,0.288261,5.372506,0.977675,329.567627,1640.829868
27,0.069297,0.348837,0.142031,0.330226,0.122658,0.551367,0.095420,0.036900,0.348090,129.0,...,0.271318,1.251708e+05,92.457364,33.333333,1.519380,0.315728,2.534884,0.819767,823.953488,5476.928203


# Algorithm

In [7]:
students = trainLabels.index.unique().tolist()
nStudents = len(students)

rd = np.random.rand(nStudents)
studentsTrain = [students[i] for i in range(nStudents) if rd[i] > 0.1]
studentsTest = [students[i] for i in range(nStudents) if rd[i] < 0.1]

In [12]:
# Building training data
X = np.array([np.array(studentsFeatures.loc[studentId].tolist(), dtype=float) for studentId in studentsTrain])
y = np.array([trainLabels['isSTEM'].loc[studentId] for studentId in studentsTrain])

# Fitting ML model
clf = linear_model.LogisticRegression()
#clf = SVC(probability=True, C=1e-1, kernel='poly')
#clf = RandomForestClassifier(max_depth=8)
clf.fit(X, y)

print('CLASSES', clf.classes_)

# Evaluating on testing data
Xt = np.array([np.array(studentsFeatures.loc[studentId].tolist(), dtype=float) for studentId in studentsTest])
yt = np.array([trainLabels['isSTEM'].loc[studentId] for studentId in studentsTest])

pred = [x[1] for x in clf.predict_proba(X)]
predT = [x[1] for x in clf.predict_proba(Xt)]

rmse = (sum([(pred[i] - y[i])**2 for i in range(len(pred))]) / len(pred))**0.5
rmseT = (sum([(predT[i] - yt[i])**2 for i in range(len(predT))]) / len(predT))**0.5

print('TRAIN', '\nACCURACY:\t', clf.score(X,y), '\nAUC:\t\t', metrics.roc_auc_score(y, pred), '\nRMSE:\t\t', rmseT, '\n')
print('TEST', '\nACCURACY:\t', clf.score(Xt,yt), '\nAUC:\t\t', metrics.roc_auc_score(yt, predT), '\nRMSE:\t\t', rmseT, '\n')

CLASSES [0 1]
TRAIN 
ACCURACY:	 0.744630071599 
AUC:		 0.64118895966 
RMSE:		 0.433155056941 

TEST 
ACCURACY:	 0.75 
AUC:		 0.574074074074 
RMSE:		 0.433155056941 



In [9]:
trainLabels['isSTEM'].loc[9]

1

In [10]:
y[16]

0